In [1]:
import requests
import pandas as pd

In [2]:
access_token = ''

In [3]:
headers = {'Authorization': 'Bearer {}'.format(access_token)}
user_url = 'https://api.spotify.com/v1/me'

r = requests.get(user_url, headers=headers)

In [4]:
user_data = r.json()
user_data.keys()

dict_keys(['country', 'display_name', 'email', 'explicit_content', 'external_urls', 'followers', 'href', 'id', 'images', 'product', 'type', 'uri'])

In [5]:
playlists_url = f'https://api.spotify.com/v1/users/{user_data["id"]}/playlists'
params = {'limit': 50}

playlists = []

while playlists_url is not None:
    r = requests.get(playlists_url, params=params, headers=headers)
    
    data = r.json()
    
    playlists.extend(data['items'])
    playlists_url = data['next']

In [6]:
names = [x['name'] for x in playlists]
start = names.index('Deal with later') + 1
end = names.index('Shazam')
names = names[start:end]

In [7]:
filtered_playlists = [playlist for playlist in playlists if playlist['name'] in names]

In [8]:
filtered_playlists[0]['tracks']

{'href': 'https://api.spotify.com/v1/playlists/7IRnBmuIaf3GfPfjLm8UDN/tracks',
 'total': 20}

In [9]:
params = {'limit': 100, 'fields': 'items(track(id)),next'}

playlist_tracks = {}

for playlist in filtered_playlists:
    tracks_url = playlist['tracks']['href']
    tracks = []
    
    while tracks_url is not None:
        r = requests.get(tracks_url, headers=headers, params=params)
        
        data = r.json()
        
        tracks.extend([x['track']['id'] for x in data['items']])
        
        tracks_url = data['next']
    
    playlist_tracks[playlist['name']] = tracks

In [10]:
playlist_tracks.keys()

dict_keys(['Breaks', 'Brostep', 'Country', 'Dance', 'Deep House', 'Disco', 'Downtempo', 'Drum n Bass', 'Drumstep', 'Dubstep', 'Dutch House', 'Electro', 'Electro House', 'Electro Swing', 'Funky House', 'G House', 'Garage', 'Glitch Hop', 'Grime', 'Hard Trance', 'Hardstyle', 'Heavy Metal', 'House', 'Instrumental Hip-Hop', 'Old Skool', 'Pop', 'Pop House', 'Progressive House', 'Psytrance', 'Rap', 'Reggae', 'Rock', 'Tech House', 'Trance', 'Trancestep', 'Trap', 'Trop House', 'Videogame'])

In [19]:
from itertools import cycle

In [20]:
playlists_df = pd.DataFrame((k, v) for k in playlist_tracks.keys() for v in playlist_tracks[k])
#playlists_df.set_index(1, inplace=True)
playlists_df.columns = ['genre', 'track_id']

In [24]:
playlists_df.to_csv('playlists-tracks.csv')